In [1]:
import numpy as np 
import pandas as pd 
import torch 
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler 
from sklearn.preprocessing import LabelEncoder

In [2]:
df = pd.read_csv("https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv")
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [3]:
df.shape

(569, 33)

In [5]:
df.drop(columns=['id', 'Unnamed: 32'], inplace=True)


# train test split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:,0], test_size=0.2)

# scaling

In [7]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# label encoding 

In [8]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

# numpy arrays to Pytorch tensors

In [9]:
X_train_tensors = torch.from_numpy(X_train)
X_test_tensors = torch.from_numpy(X_test)
y_train_tensors = torch.from_numpy(y_train)
y_test_tensors = torch.from_numpy(y_test)

In [12]:
print(X_train_tensors.shape)
print(X_train_tensors)

torch.Size([455, 30])
tensor([[ 2.2223,  0.5704,  2.2489,  ...,  1.9228, -0.2496,  0.0875],
        [-1.0111, -0.9461, -1.0331,  ..., -1.7445, -0.2986, -1.2178],
        [-0.3132,  0.7141, -0.2289,  ..., -0.0466,  1.9431,  1.2714],
        ...,
        [ 1.0770, -2.0177,  1.2443,  ...,  2.2575,  2.6909,  1.8834],
        [ 1.8177,  0.4221,  1.8619,  ...,  1.4418, -0.6623,  0.3176],
        [-0.0881, -0.6158, -0.1487,  ..., -0.8063, -0.3476, -0.3870]],
       dtype=torch.float64)


# defining the model 

In [32]:
class MySimpleNN():
    def __init__(self, X):
        self.weights = torch.rand(X.shape[1], 1, dtype=torch.float64, requires_grad=True)
        self.bias = torch.rand(1, dtype=torch.float64, requires_grad=True)

    def forward(self, X):
        z = torch.matmul(X, self.weights) + self.bias
        y_pred = torch.sigmoid(z)
        return y_pred
    
    def loss_function(self, y_pred, y):
        epsilon = 1e-7
        y_pred = torch.clamp(y_pred, epsilon, 1-epsilon)
        loss = -((y * torch.log(y_pred)) + ((1 - y) * torch.log(1 - y_pred))).mean()
        return loss

In [34]:
learning_rate = 0.1
epochs = 25

# training pipeline

In [37]:
# create model 
model = MySimpleNN(X_train_tensors)

# define loop
for epochs in range(epochs):
    # forward pass 
    y_pred = model.forward(X_train_tensors)
    # loss calculate 
    loss = model.loss_function(y_pred, y_train_tensors)
    # backward pass 
    loss.backward()
    # parameters update 
    with torch.no_grad():
        model.weights -= learning_rate * model.weights.grad
        model.bias -= learning_rate * model.bias.grad
    # zero gradient 
    model.weights.grad.zero_()
    model.bias.grad.zero_()
    # print loss in each epoch 
    print(f'Epoch: {epochs + 1}, loss:{loss.item()}')

 

Epoch: 1, loss:2.971458288419599
Epoch: 2, loss:2.8319969346804426
Epoch: 3, loss:2.6862920477072834
Epoch: 4, loss:2.5352499466188805
Epoch: 5, loss:2.3843145986274794
Epoch: 6, loss:2.23331224979853
Epoch: 7, loss:2.0838007122759183
Epoch: 8, loss:1.941591343613528
Epoch: 9, loss:1.8029273999057782
Epoch: 10, loss:1.6690713060109716
Epoch: 11, loss:1.5417403112484906
Epoch: 12, loss:1.4271228557540596
Epoch: 13, loss:1.3241724479840264
Epoch: 14, loss:1.2329837561865604
Epoch: 15, loss:1.1556944659023998
Epoch: 16, loss:1.091742863493135
Epoch: 17, loss:1.040086590162645
Epoch: 18, loss:0.9992268662976527
Epoch: 19, loss:0.967327182470469
Epoch: 20, loss:0.9424297624381336
Epoch: 21, loss:0.9227027612336167
Epoch: 22, loss:0.9066218190020359
Epoch: 23, loss:0.893037568147554


In [43]:
# model evaluation 
with torch.no_grad():
    y_pred = model.forward(X_test_tensors)
    y_pred = (y_pred>0.5).float()

    acc = (y_pred == y_test_tensors).float().mean()
    print(acc)

tensor(0.4346)
